# Generate Sigma Points for Asteroids
This notebook demonstrates how to query JPL CNEOS API for orbital elements and covariances and generate sigma points from that covariance matrix. Those sigma points are then converted to Cartesian states that can be used for uncertainty propagation.

S. Eggl 20200123

In [1]:
import numpy as np
import jpl_query
from scipy.linalg import cholesky

In [2]:
def covariance2sigmapoints(mean,cov,h=np.sqrt(3)):
    """Generate sigma points from covariance matrix based on +-Sqrt(nM)
    
    Parameters: 
    -----------
    mean ... vector of mean values (dimension n)
    cov ... covariance matrix
    
    Returns:
    --------
    sigmap ... list of sigma points (2n+1, dimension n)
    """

    # Lower triangular matrix such that P = Sxx * transpose(Sxx)
    x=np.array(mean)
    sqrt_xx = cholesky(np.array(cov), lower=True)
    sig_pts = []
    sig_pts.append(x)
    #print(sqrt_xx)
    for col in sqrt_xx.T:  # loop over columns
        sig_pts.append(x + (h * col))
        sig_pts.append(x - (h * col))
    
    sigmap=np.array(sig_pts)
    #do a quick unit test here
    spmean=np.mean(sigmap,axis=0)
    spcov=np.cov(sigmap.T/h,bias=True)
    #print(spcov)
    dmean=spmean-mean
    if(np.dot(dmean,dmean)>1e-13):
        raise ValueError('Error: sigma point mean is not close to original mean')
    
    if(np.sum(np.subtract(spcov,cov))>1e-13):
        raise ValueError('Error: sigma point covariance is not close to original covariance')
    return sigmap

In [3]:
def sigmapoints2covariance(p,h=np.sqrt(3)):
    """Takes a numpy array of vectors and calulates the covariance matrix 'manually'
    
    Parameters:
    -----------
    p ... array of input vectors
    
    Returns:
    --------
    pmean ... mean
    cov ... covariance matrix
    """
    pmean=np.mean(p,axis=0)
    n=p.shape[0]
    #recreate the covariance matrix from sigma points
    #the factor of n/2 comes from the number of sigma points (we don't want the covariance matrix divided by that number)
    # and the +- distances from the mean 
    cov=np.cov(p.T/h,bias=True)*n/2
    #cov=np.cov(p.T/h,bias=False)
    return pmean, cov

In [4]:
#TARGET ASTEROID IDENTIFIER FOR JPL QUERY
tname='Duende'

In [5]:
#QUERY JPL FOR ORBITAL ELEMENTS AND COVARIANCE MATRIX
state=jpl_query.query_cometary_ele_and_cov(tname)

In [6]:
#EPOCH FOR ORBITAL ELEMENTS AND COVARIANCE MATRIX (JD)
epoch=state[0]

In [7]:
#COMETARY ORBITAL ELEMENTS [e,q,tp(JD),node(deg),peri(deg),inc(deg)]
com=np.array(state[1])

In [8]:
com

array([1.08129352e-01, 8.93492625e-01, 2.45626155e+06, 1.47262479e+02,
       2.71086261e+02, 1.03372996e+01])

In [9]:
#COVARIANCE MATRIX
cov=np.array(state[2])

In [10]:
cov

array([[ 1.22427066e-15, -1.21595471e-15, -7.70335932e-15,
         7.27276618e-16, -1.49591848e-13, -3.61762539e-14],
       [-1.21595471e-15,  1.21014550e-15,  8.78586162e-15,
        -7.17810854e-16,  1.50033328e-13,  3.59520167e-14],
       [-7.70335932e-15,  8.78586162e-15,  6.00186647e-13,
         5.81889730e-15,  1.63466838e-12,  2.85535705e-13],
       [ 7.27276618e-16, -7.17810854e-16,  5.81889730e-15,
         1.11320495e-14, -8.87319114e-14,  3.01011899e-14],
       [-1.49591848e-13,  1.50033328e-13,  1.63466838e-12,
        -8.87319114e-14,  1.91672708e-11,  4.42994470e-12],
       [-3.61762539e-14,  3.59520167e-14,  2.85535705e-13,
         3.01011899e-14,  4.42994470e-12,  1.33606512e-12]])

## Create Sigma Points from Covariance Matrix

In [11]:
sigma_points=covariance2sigmapoints(com,cov)

In [12]:
sigma_points

array([[1.08129352e-01, 8.93492625e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086261e+02, 1.03372996e+01],
       [1.08129413e-01, 8.93492564e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086254e+02, 1.03372978e+01],
       [1.08129292e-01, 8.93492685e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086269e+02, 1.03373014e+01],
       [1.08129352e-01, 8.93492627e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086263e+02, 1.03372996e+01],
       [1.08129352e-01, 8.93492622e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086260e+02, 1.03372996e+01],
       [1.08129352e-01, 8.93492625e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086262e+02, 1.03373001e+01],
       [1.08129352e-01, 8.93492625e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086261e+02, 1.03372991e+01],
       [1.08129352e-01, 8.93492625e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086261e+02, 1.03373003e+01],
       [1.08129352e-01, 8.93492625e-01, 2.45626155e+06, 1.47262479e+02,
        2.71086262e+02, 

### Test if Sigma Points reproduce original Covariance Matrix

In [13]:
[pmean,cov_new]=sigmapoints2covariance(sigma_points)

In [14]:
#Calculate the difference between original and Sigma Point Covariance. Small is good.
cov/cov_new

array([[1.        , 1.        , 0.99956297, 0.99999964, 1.        ,
        1.        ],
       [1.        , 1.        , 0.99960908, 0.99999965, 1.        ,
        1.        ],
       [0.99956297, 0.99960908, 0.99981176, 1.00072124, 0.99971849,
        0.99969658],
       [0.99999964, 0.99999965, 1.00072124, 1.00000013, 0.99999969,
        1.00000039],
       [1.        , 1.        , 0.99971849, 0.99999969, 1.        ,
        1.        ],
       [1.        , 1.        , 0.99969658, 1.00000039, 1.        ,
        1.        ]])

## Convert Sigma Points from Cometary to Cartesian

In [15]:
#Epoch
state[0]

2456165.5

In [16]:
sp_cart=[]
for s in sigma_points:
    sp_cart.append(jpl_query.cometary2cartesian(state[0],s))

In [17]:
#List of Cartesian states of Sigma points
sp_cart

[array([ 0.68329937, -0.75783578,  0.0488725 ,  0.01115044,  0.01227682,
        -0.00298354]),
 array([ 0.68329921, -0.75783595,  0.04887254,  0.01115044,  0.01227682,
        -0.00298353]),
 array([ 0.68329954, -0.75783561,  0.04887247,  0.01115044,  0.01227683,
        -0.00298354]),
 array([ 0.68329939, -0.75783577,  0.0488725 ,  0.01115044,  0.01227683,
        -0.00298354]),
 array([ 0.68329936, -0.75783578,  0.04887251,  0.01115044,  0.01227682,
        -0.00298354]),
 array([ 0.68329938, -0.75783578,  0.04887251,  0.01115044,  0.01227682,
        -0.00298354]),
 array([ 0.68329937, -0.75783578,  0.0488725 ,  0.01115044,  0.01227682,
        -0.00298353]),
 array([ 0.68329937, -0.75783578,  0.04887251,  0.01115044,  0.01227682,
        -0.00298354]),
 array([ 0.68329937, -0.75783578,  0.0488725 ,  0.01115044,  0.01227682,
        -0.00298353]),
 array([ 0.68329937, -0.75783578,  0.0488725 ,  0.01115044,  0.01227682,
        -0.00298354]),
 array([ 0.68329937, -0.75783578,  0.048